In [25]:
from transformers import pipeline

In [26]:
sentiment_pipeline = pipeline("sentiment-analysis", device=0)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [4]:
import torch
torch.cuda.is_available()

True

In [17]:
# import pandas as pd

# data = pd.read_csv("movies.csv")
# reviews = data["review_title"]
# result = [0 for _ in range(250)]
# counter = 0
# for i in range(len(data)):
#     result[counter] = sentiment_pipeline(reviews[i])[0]
#     counter += 1
#     if counter % 1000 == 0:
#         print(counter // 1000)
# result

c:\Users\Yang\.conda\envs\datamine\lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[{'label': 'POSITIVE', 'score': 0.9997935891151428},
 {'label': 'POSITIVE', 'score': 0.9998335838317871},
 {'label': 'POSITIVE', 'score': 0.99956876039505},
 {'label': 'POSITIVE', 'score': 0.7947134375572205},
 {'label': 'POSITIVE', 'score': 0.999325156211853},
 {'label': 'POSITIVE', 'score': 0.9997863173484802},
 {'label': 'POSITIVE', 'score': 0.9998562335968018},
 {'label': 'POSITIVE', 'score': 0.9997828602790833},
 {'label': 'POSITIVE', 'score': 0.9998043179512024},
 {'label': 'POSITIVE', 'score': 0.9998295307159424},
 {'label': 'POSITIVE', 'score': 0.9997842907905579},
 {'label': 'POSITIVE', 'score': 0.999204695224762},
 {'label': 'POSITIVE', 'score': 0.9998291730880737},
 {'label': 'POSITIVE', 'score': 0.9990284442901611},
 {'label': 'POSITIVE', 'score': 0.9998427629470825},
 {'label': 'POSITIVE', 'score': 0.9998512268066406},
 {'label': 'POSITIVE', 'score': 0.9997567534446716},
 {'label': 'POSITIVE', 'score': 0.9992086291313171},
 {'label': 'POSITIVE', 'score': 0.9995748400688171

In [15]:
from datasets import load_dataset
imdb = load_dataset("imdb")

In [16]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis", device=0)
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])
sentiment_pipeline(small_test_dataset)


In [35]:
from transformers import pipeline
from datasets import load_dataset

# Load IMDb dataset
imdb = load_dataset("imdb")

# Specify GPU device index (change to 0, 1, etc., based on your GPU setup)
gpu_device_index = 0

# Create sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", device=gpu_device_index)

# Select a subset of the IMDb dataset for testing
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in range(300)])

# Get the maximum sequence length expected by the model
max_seq_length = sentiment_pipeline.model.config.max_position_embeddings

# Perform sentiment analysis on the selected subset
results = sentiment_pipeline([{"text": text[:max_seq_length]} for text in small_test_dataset["text"]])

# Print the results
print(results)


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9975194334983826}, {'label': 'POSITIVE', 'score': 0.9980888962745667}, {'label': 'NEGATIVE', 'score': 0.9853643774986267}, {'label': 'POSITIVE', 'score': 0.9979335069656372}, {'label': 'NEGATIVE', 'score': 0.9985034465789795}, {'label': 'POSITIVE', 'score': 0.9998026490211487}, {'label': 'POSITIVE', 'score': 0.9959130883216858}, {'label': 'NEGATIVE', 'score': 0.9995379447937012}, {'label': 'NEGATIVE', 'score': 0.9969226717948914}, {'label': 'POSITIVE', 'score': 0.9998483657836914}, {'label': 'POSITIVE', 'score': 0.9998674392700195}, {'label': 'NEGATIVE', 'score': 0.9989897608757019}, {'label': 'NEGATIVE', 'score': 0.9991999268531799}, {'label': 'NEGATIVE', 'score': 0.9986159801483154}, {'label': 'POSITIVE', 'score': 0.9993453621864319}, {'label': 'POSITIVE', 'score': 0.9997767806053162}, {'label': 'POSITIVE', 'score': 0.9028362035751343}, {'label': 'POSITIVE', 'score': 0.9350964426994324}, {'label': 'POSITIVE', 'score': 0.9997335076332092}, {'label': '

In [54]:
from transformers import pipeline
from datasets import load_dataset
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords

# Load IMDb dataset
imdb = load_dataset("imdb")

# Specify GPU device index (change to 0, 1, etc., based on your GPU setup)
gpu_device_index = 0

# Create sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", device=gpu_device_index)

# Select a subset of the IMDb dataset for testing
small_test_dataset = imdb["train"].shuffle(seed=42).select([i for i in range(10000)])

# Get the maximum sequence length expected by the model
max_seq_length = sentiment_pipeline.model.config.max_position_embeddings

# Perform sentiment analysis on the selected subset
results = sentiment_pipeline([{"text": text[:max_seq_length]} for text in small_test_dataset["text"]])

# Extract negative sentiment texts
negative_texts = [text["text"] for text, result in zip(small_test_dataset, results) if result["label"] == "POSITIVE"]

# Combine all negative texts into a single string
all_negative_text = " ".join(negative_texts)

# Perform basic text preprocessing (lowercasing and removing non-alphanumeric characters)
all_negative_text = re.sub(r"[^a-zA-Z0-9\s]", "", all_negative_text.lower())

# Tokenize the text and filter out stop words
stop_words = set(stopwords.words("english"))
bad_words = ["br", "movies", "movie", "film", "films", "one"]
for i in bad_words:
    stop_words.add(i)
tokens = [word for word in all_negative_text.split() if word not in stop_words]

# Find common meaningful patterns
pos_common_patterns = Counter(tokens).most_common(25)

# Print the most common patterns
print("Most common meaningful patterns in negative sentiment:")
for pattern, count in common_patterns:
    print(f"{pattern}: {count} occurrences")

# Extract negative sentiment texts
negative_texts = [text["text"] for text, result in zip(small_test_dataset, results) if result["label"] == "NEGATIVE"]

# Combine all negative texts into a single string
all_negative_text = " ".join(negative_texts)

# Perform basic text preprocessing (lowercasing and removing non-alphanumeric characters)
all_negative_text = re.sub(r"[^a-zA-Z0-9\s]", "", all_negative_text.lower())

tokens = [word for word in all_negative_text.split() if word not in stop_words]

# Find common meaningful patterns
neg_common_patterns = Counter(tokens).most_common(25)

# Print the most common patterns
print("Most common meaningful patterns in negative sentiment:")
for pattern, count in pos_common_patterns:
    print(f"{pattern}: {count} occurrences")
for pattern, count in neg_common_patterns:
    print(f"{pattern}: {count} occurrences")


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Most common meaningful patterns in negative sentiment:
like: 4500 occurrences
even: 3071 occurrences
good: 2980 occurrences
would: 2835 occurrences
bad: 2785 occurrences
really: 2533 occurrences
time: 2442 occurrences
see: 2325 occurrences
get: 2180 occurrences
dont: 2143 occurrences
story: 2108 occurrences
much: 2099 occurrences
people: 1995 occurrences
could: 1946 occurrences
make: 1915 occurrences
made: 1888 occurrences
first: 1871 occurrences
well: 1701 occurrences
also: 1663 occurrences
way: 1601 occurrences
acting: 1572 occurrences
think: 1570 occurrences
plot: 1543 occurrences
characters: 1476 occurrences
watch: 1443 occurrences
Most common meaningful patterns in negative sentiment:
like: 3334 occurrences
good: 2877 occurrences
story: 2427 occurrences
great: 2231 occurrences
see: 2208 occurrences
time: 2141 occurrences
well: 2141 occurrences
really: 2046 occurrences
would: 2018 occurrences
also: 1933 occurrences
even: 1925 occurrences
much: 1768 occurrences
first: 1648 occurrenc

In [62]:
import pandas as pd
empty_list = [0 for _ in range(25)]
result = pd.DataFrame({"Pos Word": empty_list, "Pos Count": empty_list, "Neg Word": empty_list, "Neg Count": empty_list})

for i in range(25):
    result.iloc[i,:] = [pos_common_patterns[i][0], pos_common_patterns[i][1], neg_common_patterns[i][0], neg_common_patterns[i][1]]

result

,Pos Word,Pos Count,Neg Word,Neg Count
0,like,3334,like,4500
1,good,2877,even,3071
2,story,2427,good,2980
3,great,2231,would,2835
4,see,2208,bad,2785
5,time,2141,really,2533
6,well,2141,time,2442
7,really,2046,see,2325
8,would,2018,get,2180
9,also,1933,dont,2143
